In [1]:
import os
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
# from transformers import GPT2LMHeadModel, GPT2Tokenizer

from utils import *; from boring_utils.utils import *

init_graph()
set_seed(42)
device = get_device()

In [2]:
# since we gonna do some visualization, let's put it on cpu
model_type = 'distilgpt2'
# model_type = 'gpt2'

model = AutoModel.from_pretrained(model_type, output_attentions=True)
model_clm = AutoModelForCausalLM.from_pretrained(model_type)
tokenizer_hf = AutoTokenizer.from_pretrained(model_type)

# HF Export Attention Weights

In [3]:
# tensor([[ 464, 3797, 3332,  319,  262, 2603]])
inputs = tokenizer_hf.encode("The cat sat on the mat", return_tensors='pt')

In [4]:
outputs = model(
        inputs,
    )

`GPT2SdpaAttention` is used but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


In [5]:
outputs_dict = model_clm.generate(
        inputs, 
        output_attentions=True,  # we need to add this line to get the attention
        return_dict_in_generate=True,
        use_cache = True,
    )

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/Users/tim/miniforge3/envs/torch/lib/python3.11/site-packages/transformers/generation/utils.py:1259: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Compare AutoModel(inputs) vs AutoModelForCausalLM.generate(inputs) and their attention masks

In [6]:
type(outputs), type(outputs_dict)

(transformers.modeling_outputs.BaseModelOutputWithPastAndCrossAttentions,
 transformers.generation.utils.GenerateDecoderOnlyOutput)

In [7]:
cprint(outputs.keys(), outputs_dict.keys())

<module> -> outputs.keys():
odict_keys(['last_hidden_state', 'past_key_values', 'attentions'])
<module> -> outputs_dict.keys():
odict_keys(['sequences', 'attentions', 'past_key_values'])


NOTE: Since there is no "generation head" for AutoModel, so the "attentions" here is more like the embedding purpuse, i.e. the input embedding.

In [8]:
# outputs[-1] == outputs['attentions']  # True
# outputs['attentions'][0] == outputs_dict['attentions'][0][0]  # True
# outputs['attentions'][0][0] == outputs_dict['attentions'][0][0].squeeze(0)  # True

In [ ]:
cprint(len(inputs))
cprint(outputs_dict["attentions"][0][0].shape, outputs_dict["attentions"][0][-1].shape)
cprint(outputs_dict["attentions"][-1][0].shape, outputs_dict["attentions"][-1][-1].shape)

# Bertviz Head View

```
pip install bertviz
```

In [10]:
# from bertviz import head_view

# attention = outputs[-1]  # Output includes attention weights when output_attentions=True
# tokens = tokenizer_hf.convert_ids_to_tokens(inputs[0])
# head_view(attention, tokens)